This notebook is associated with the Appendix of the paper ["A fundamental non-classical logic"](https://arxiv.org/abs/2207.06993) by [Wesley H. Holliday](mailto:wesholliday@berkeley.edu).

The notebook uses the [Natural Language Toolkit](https://www.nltk.org)'s [interface](https://www.nltk.org/howto/inference.html) to [Prover9/Mace4](https://www.cs.unm.edu/~mccune/prover9/) to verify basic facts about the kinds of implications discussed in the Appendix of the paper.

To view the notebook online, type the URL of this notebook (https://github.com/wesholliday/fundamental-logic/blob/main/implication-axioms.ipynb) into the location field at https://nbviewer.org. GitHub's preview of the notebook might not show all the output that was generated.

## Outline

**1. [Lattice axioms](#1)**

**2. [Bounded lattice axioms](#2)**

**3. [Implication axioms](#3)**

In [1]:
from nltk.test.inference_fixt import setup_module 

setup_module()

from nltk import *
from nltk.sem.drt import DrtParser
from nltk.sem import logic
logic._counter._value = 0

from nltk.sem import Expression
read_expr = Expression.fromstring

## 1. Lattice axioms<a id='1'></a>

In [2]:
or_id = read_expr('Or(x,x) = x')
and_id = read_expr('And(x,x) = x')

or_comm = read_expr('Or(x,y) = Or(y,x)')
and_comm = read_expr('And(x,y) = And(y,x)')

or_assoc = read_expr('Or(x,Or(y,z)) = Or(Or(x,y),z)')
and_assoc = read_expr('And(x,And(y,z)) = And(And(x,y),z)')

or_absorp = read_expr('Or(x,And(x,y)) = x')
and_absorp = read_expr('And(x,Or(x,y)) = x')

lattice = [or_id, and_id, or_comm, and_comm, or_assoc, and_assoc, or_absorp, and_absorp]

In [3]:
#By including a definition of the covering relation,
#one can quickly read off Hasse diagrams of lattices from the Mace4 output.
cover_def = read_expr('covered_by(x,y) <-> (-(x=y) & x = And(x,y) & -exists z.(-(z=x) & -(z=y) & x = And(x,z) & z = And(z,y)))')

## 2. Bounded lattice axioms<a id='2'></a>

In [4]:
bot = read_expr('Or(x,Bot) = x')
top = read_expr('And(x,Top) = x')

bounded_lattice = lattice + [bot,top]

## 3. Implication axioms<a id='3'></a>

## Preimplications

In [5]:
preimp1_left_to_right = read_expr('x = And(x,If(Top,x))')
preimp1_right_to_left = read_expr('If(Top,x) = And(If(Top,x),x)')
preimp2 = read_expr('If(x,If(x,y)) = And(If(x,If(x,y)), If(x,y))')
preimp3 = read_expr('x = And(x,y) -> If(y,z) = And(If(y,z), If(x,z))')
preimp4 = read_expr('x = And(x,y) -> If(z,x) = And(If(z,x), If(z,y))')

preimp = [preimp1_left_to_right, preimp1_right_to_left, preimp2, preimp3, preimp4]

In [6]:
#Verify that the preimplication axioms are independent

for ax in preimp:
    print(f"Independence of {ax}: \n")
    goal = ax
    mb = MaceCommand(goal, assumptions = bounded_lattice + [axiom for axiom in preimp if not axiom == ax])
    mb.build_model()
    print(mb.model(format='cooked'))

Independence of (x = And(x,If(Top,x))): 

% number = 1
% seconds = 0

% Interpretation of size 2

Bot = 0.

Top = 1.

c1 = 1.

And(0,0) = 0.
And(0,1) = 0.
And(1,0) = 0.
And(1,1) = 1.

If(0,0) = 0.
If(0,1) = 0.
If(1,0) = 0.
If(1,1) = 0.

Or(0,0) = 0.
Or(0,1) = 1.
Or(1,0) = 1.
Or(1,1) = 1.

Independence of (If(Top,x) = And(If(Top,x),x)): 

% number = 1
% seconds = 0

% Interpretation of size 2

Bot = 0.

Top = 1.

c1 = 0.

And(0,0) = 0.
And(0,1) = 0.
And(1,0) = 0.
And(1,1) = 1.

If(0,0) = 1.
If(0,1) = 1.
If(1,0) = 1.
If(1,1) = 1.

Or(0,0) = 0.
Or(0,1) = 1.
Or(1,0) = 1.
Or(1,1) = 1.

Independence of (If(x,If(x,y)) = And(If(x,If(x,y)),If(x,y))): 

% number = 1
% seconds = 0

% Interpretation of size 3

Bot = 0.

Top = 1.

c1 = 0.

c2 = 0.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.

If(0,0) = 2.
If(0,1) = 1.
If(0,2) = 1.
If(1,0) = 0.
If(1,1) = 1.
If(1,2) = 2.
If(2,0) = 0.
If(2,1) = 1.
If(2,2) = 1.

Or(0,0) =

In [7]:
#Verify that every bounded lattice can be equipped with a preimplication using the following definition

preimp_def = read_expr('(x=And(x,y) -> If(x,y)=Top) & (-(x=And(x,y)) -> If(x,y)=y)')

for ax in preimp:
    print(f"Proof of {ax}")
    goal = ax
    prover = Prover9Command(goal, assumptions = bounded_lattice + [preimp_def])
    prover.prove()
    print(prover.proof())

Proof of (x = And(x,If(Top,x)))
============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 24391 was started by halcrow on ras-26-185.vpn.berkeley.edu,
Wed Sep  7 19:20:12 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 12.
% Level of proof is 3.
% Maximum clause weight is 10.000.
% Given clauses 13.


1 (x = And(x,y) -> If(x,y) = Top) & (x != And(x,y) -> If(x,y) = y).  [assumption].
2 x = And(x,If(Top,x)).  [goal].
4 And(x,x) = x.  [assumption].
6 And(x,y) = And(y,x).  [assumption].
14 And(x,Top) = x.  [assumption].
15 And(x,y) != x | If(x,y) = Top.  [clausify(1)].
16 And(x,y) = x | If(x,y) = y.  

In [8]:
#Verify that given an orthocomplementation Not, the implication If(x,y) defined by Not(x and Not(y)) is a preimplication.

antitone = read_expr('x=And(x,y) -> Not(y)=And(Not(x),Not(y))')
semicomp = read_expr('And(x,Not(x)) = Bot')
DNI = read_expr('x=And(x,Not(Not(x)))')
DNE = read_expr('Not(Not(x))=And(Not(Not(x)),x)')
LEM = read_expr('Or(x,Not(x)) = Top')
orthocomp = [antitone, semicomp, DNI, DNE, LEM]

ortho_preimp_def = read_expr('If(x,y) = Not(And(x,Not(y)))')

for ax in preimp:
    print(f"Proof of {ax}")
    goal = ax
    prover = Prover9Command(goal, assumptions = bounded_lattice + orthocomp + [ortho_preimp_def])
    prover.prove()
    print(prover.proof())

Proof of (x = And(x,If(Top,x)))
============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 24401 was started by halcrow on ras-26-185.vpn.berkeley.edu,
Wed Sep  7 19:20:12 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 13.
% Level of proof is 3.
% Maximum clause weight is 9.000.
% Given clauses 10.


2 x = And(x,If(Top,x)).  [goal].
4 And(x,x) = x.  [assumption].
6 And(x,y) = And(y,x).  [assumption].
14 And(x,Top) = x.  [assumption].
18 x = And(x,Not(Not(x))).  [assumption].
19 And(x,Not(Not(x))) = x.  [copy(18),flip(a)].
20 Not(Not(x)) = And(Not(Not(x)),x).  [assumption].
21 Not(Not(x)) = x.  [co

## Protoimplications

In [9]:
identity = read_expr('If(x,x)=Top')
modus_ponens = read_expr('And(x,If(x,y))= And(And(x,If(x,y)),y)')

protoimp = preimp + [identity,modus_ponens]

In [10]:
#Verify that the identity axiom does not already follow from preimplication axioms

goal = identity
mb = MaceCommand(goal, assumptions = bounded_lattice + preimp + [modus_ponens])
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 2

Bot = 0.

Top = 1.

c1 = 0.

And(0,0) = 0.
And(0,1) = 0.
And(1,0) = 0.
And(1,1) = 1.

If(0,0) = 0.
If(0,1) = 1.
If(1,0) = 0.
If(1,1) = 1.

Or(0,0) = 0.
Or(0,1) = 1.
Or(1,0) = 1.
Or(1,1) = 1.



In [11]:
#Verify that modus ponens does not already follow from preimplication axioms

goal = modus_ponens
mb = MaceCommand(goal, assumptions = bounded_lattice + preimp + [identity])
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 3

Bot = 0.

Top = 1.

c1 = 2.

c2 = 0.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.

If(0,0) = 1.
If(0,1) = 1.
If(0,2) = 1.
If(1,0) = 0.
If(1,1) = 1.
If(1,2) = 2.
If(2,0) = 1.
If(2,1) = 1.
If(2,2) = 1.

Or(0,0) = 0.
Or(0,1) = 1.
Or(0,2) = 2.
Or(1,0) = 1.
Or(1,1) = 1.
Or(1,2) = 1.
Or(2,0) = 2.
Or(2,1) = 1.
Or(2,2) = 2.



## Ultraweak pseudoimplications

In [12]:
ultraweak_pseudo_ax = read_expr('x = And(x, If(If(x,y), y))')
ultraweak_pseudo_ax_equiv = read_expr('x=And(x,If(z,y)) -> z=And(z,If(x,y))')

ultraweak_pseudo = preimp + [ultraweak_pseudo_ax]

In [13]:
# Verify Lemma A.5 from 1 to 2

goal = ultraweak_pseudo_ax_equiv
prover = Prover9Command(goal, assumptions = bounded_lattice + preimp + [ultraweak_pseudo_ax])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 24415 was started by halcrow on ras-26-185.vpn.berkeley.edu,
Wed Sep  7 19:20:13 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.13 (+ 0.01) seconds.
% Length of proof is 16.
% Level of proof is 5.
% Maximum clause weight is 16.000.
% Given clauses 223.


1 x = And(x,y) -> If(y,z) = And(If(y,z),If(x,z)).  [assumption].
3 x = And(x,If(y,z)) -> y = And(y,If(x,z)).  [goal].
7 And(x,y) = And(y,x).  [assumption].
10 And(x,And(y,z)) = And(And(x,y),z).  [assumption].
11 And(And(x,y),z) = And(x,And(y,z)).  [copy(10),flip(a)].
22 And(x,y) != x | If(y,z) = And(If(y,z),If(x,z)).  [clausify(1)

In [14]:
# Verify Lemma A.5 from 2 to 1

goal = ultraweak_pseudo_ax
prover = Prover9Command(goal, assumptions = bounded_lattice + preimp + [ultraweak_pseudo_ax_equiv])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 24417 was started by halcrow on ras-26-185.vpn.berkeley.edu,
Wed Sep  7 19:20:13 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 7.
% Level of proof is 3.
% Maximum clause weight is 14.000.
% Given clauses 15.


3 x = And(x,If(y,z)) -> y = And(y,If(x,z)).  [assumption].
4 x = And(x,If(If(x,y),y)).  [goal].
6 And(x,x) = x.  [assumption].
27 And(x,If(y,z)) != x | And(y,If(x,z)) = y.  [clausify(3)].
28 And(c1,If(If(c1,c2),c2)) != c1.  [deny(4)].
61 And(x,If(If(x,y),y)) = x.  [hyper(27,a,6,a)].
62 $F.  [resolve(61,a,28,a)].

===================

In [15]:
#Verify that the ultraweak pseudo axiom does not already follow from the preimplication axioms

goal = ultraweak_pseudo_ax
mb = MaceCommand(goal, assumptions = bounded_lattice + preimp)
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 2

Bot = 0.

Top = 1.

c1 = 1.

c2 = 0.

And(0,0) = 0.
And(0,1) = 0.
And(1,0) = 0.
And(1,1) = 1.

If(0,0) = 0.
If(0,1) = 1.
If(1,0) = 0.
If(1,1) = 1.

Or(0,0) = 0.
Or(0,1) = 1.
Or(1,0) = 1.
Or(1,1) = 1.



## Weak pseudoimplications

In [16]:
weak_pseudoimp = protoimp + [ultraweak_pseudo_ax]

In [17]:
relative_pseudo_left_to_right = read_expr('And(x,y)=And(And(x,y),z) -> x=And(x,If(y,z))')

In [18]:
#Verify that the left-to-right direction of the relative pseudocomplementation principle 
#does not follow from the weak pseudoimplication axioms

goal = relative_pseudo_left_to_right
mb = MaceCommand(goal, assumptions = weak_pseudoimp)
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 3

Top = 0.

c1 = 0.

c2 = 1.

c3 = 2.

And(0,0) = 0.
And(0,1) = 2.
And(0,2) = 2.
And(1,0) = 1.
And(1,1) = 1.
And(1,2) = 2.
And(2,0) = 2.
And(2,1) = 2.
And(2,2) = 2.

If(0,0) = 0.
If(0,1) = 1.
If(0,2) = 2.
If(1,0) = 0.
If(1,1) = 0.
If(1,2) = 2.
If(2,0) = 0.
If(2,1) = 0.
If(2,2) = 0.



In [20]:
#Verify that the preimplication defined for any bounded lattice above is in fact a weak pseudoimplication

for ax in weak_pseudoimp:
    print(f"Proof of {ax}")
    goal = ax
    prover = Prover9Command(goal, assumptions = bounded_lattice + [preimp_def])
    prover.prove()
    print(prover.proof())

Proof of (x = And(x,If(Top,x)))
============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 24479 was started by halcrow on ras-26-185.vpn.berkeley.edu,
Wed Sep  7 19:20:52 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 12.
% Level of proof is 3.
% Maximum clause weight is 10.000.
% Given clauses 13.


1 (x = And(x,y) -> If(x,y) = Top) & (x != And(x,y) -> If(x,y) = y).  [assumption].
2 x = And(x,If(Top,x)).  [goal].
4 And(x,x) = x.  [assumption].
6 And(x,y) = And(y,x).  [assumption].
14 And(x,Top) = x.  [assumption].
15 And(x,y) != x | If(x,y) = Top.  [clausify(1)].
16 And(x,y) = x | If(x,y) = y.  